In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras import applications
from keras.models import Sequential, Model, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Conv2D, MaxPooling2D, LSTM, Reshape, Permute, TimeDistributed, CuDNNLSTM,LeakyReLU
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
train_path = '/home/livelab/Desktop/NewResizedBalanced5000/Training/' # directory containing subsets of data with labels
valid_path = '/home/livelab/Desktop/NewResized/Validation/'
test_path = '/home/livelab/Desktop/NewResized/Test/'

In [3]:
def findLayerNumByName(model, name):
    layerList=model.layers
    return layerList.index(model.get_layer(layer_name))

In [5]:
BATCH_SIZE = 64 # how many to process at once (greatest power of 2 that can ft in RAM)

train_datagen = keras.preprocessing.image.ImageDataGenerator(zca_whitening=True, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0, height_shift_range=0.0, brightness_range=None, 
                                                             shear_range=0.0, zoom_range=0.0, channel_shift_range=0.0, fill_mode='nearest', 
                                                             cval=0.0, horizontal_flip=True, vertical_flip=True)
valid_datagen = keras.preprocessing.image.ImageDataGenerator()
test_datagen = keras.preprocessing.image.ImageDataGenerator()

# Use the ImageDataGenerators to load the training data
train_gen = train_datagen.flow_from_directory(directory=train_path,
                                    target_size=(256,256), # size to resize images to
                                    color_mode='rgb', # color mode of the images
                                    batch_size=BATCH_SIZE, # how many images to process at once
                                    class_mode='categorical', # classify into categorical classes
                                    shuffle=True # shuffle order of images
)
valid_gen = valid_datagen.flow_from_directory(directory=valid_path,
                                    target_size=(256,256),
                                    color_mode='rgb',
                                    batch_size=BATCH_SIZE,
                                    class_mode='categorical',
                                    shuffle=True
)
test_gen = test_datagen.flow_from_directory(directory=test_path,
                                    target_size=(256,256),
                                    color_mode='rgb',
                                    batch_size=BATCH_SIZE,
                                    class_mode='categorical',
                                    shuffle=False
)

/home/livelab/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 119337 images belonging to 24 classes.
Found 163247 images belonging to 24 classes.
Found 171954 images belonging to 24 classes.


In [4]:
#full inception
#freeze mix3 and prior
inputData=Input(shape=(256, 256, 3))
model = applications.inception_v3.InceptionV3(weights='imagenet', include_top=False,input_shape = ( 256, 256, 3))
layer_name = 'mixed6'
index=findLayerNumByName(model, layer_name)
intermediate_model=Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
# for layer in intermediate_model.layers:
#     layer.trainable = False
intermediateOut = Flatten()(intermediate_model.output)
intermediateOut = Dense(32)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = LeakyReLU(alpha=0.1)(intermediateOut)
intermediateOut = Dropout(0.4)(intermediateOut)
prediction = Dense(24, activation='softmax')(intermediateOut)
optmr = Adam(lr=0.0005, decay = 5e-4)
model_final = Model(input = model.input, output = prediction)
model_final.compile(loss = "categorical_crossentropy", optimizer = optmr, metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/livelab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [ ]:
model_final.summary()

In [6]:
filepath = "model_partial-x5-5000-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,save_weights_only=True, mode='auto', period=1)
callbacks_list = [checkpoint]

In [7]:
STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size/20
STEP_SIZE_VALID=valid_gen.n//valid_gen.batch_size

In [8]:
with tf.device('/GPU:0'):
    model_final.fit_generator(generator=train_gen,
                        steps_per_epoch=STEP_SIZE_TRAIN, # number of steps in each epoch
                        validation_data=valid_gen,
                        validation_steps=STEP_SIZE_VALID,
                        callbacks=callbacks_list,
                        epochs=40
    )

Instructions for updating:
Use tf.cast instead.
Epoch 1/40


/home/livelab/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/livelab/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


94/93 [==============================] - 346s 4s/step - loss: 2.5736 - acc: 0.2694 - val_loss: 2.6280 - val_acc: 0.3881

Epoch 00001: val_acc improved from -inf to 0.38809, saving model to model_partial-x5-5000-01-0.39.hdf5
Epoch 2/40
94/93 [==============================] - 340s 4s/step - loss: 1.7347 - acc: 0.5332 - val_loss: 2.8038 - val_acc: 0.1636

Epoch 00002: val_acc did not improve from 0.38809
Epoch 3/40
94/93 [==============================] - 341s 4s/step - loss: 1.4274 - acc: 0.6336 - val_loss: 2.5591 - val_acc: 0.3256

Epoch 00003: val_acc did not improve from 0.38809
Epoch 4/40
94/93 [==============================] - 341s 4s/step - loss: 1.2042 - acc: 0.6973 - val_loss: 2.7479 - val_acc: 0.1692

Epoch 00004: val_acc did not improve from 0.38809
Epoch 5/40
94/93 [==============================] - 341s 4s/step - loss: 1.0266 - acc: 0.7434 - val_loss: 2.2833 - val_acc: 0.4265

Epoch 00005: val_acc improved from 0.38809 to 0.42648, saving model to model_partial-x5-5000-05-0.

In [5]:
model_final.load_weights("model_partial-x5-5000-08-0.47.hdf5")

In [11]:
STEP_SIZE_TEST=test_gen.n//test_gen.batch_size
# evaluate the function using the test set
test_loss, test_acc = model_final.evaluate_generator(test_gen,
                                            STEP_SIZE_TEST)

# print test accuracy
print('Accuracy: {}'.format(test_acc))

Accuracy: 0.4454069463028681


In [12]:
#compute confusion ma14trix
#col : truth, #row : pred
conf=np.zeros((24,24))
STEP_SIZE_TEST=test_gen.n//test_gen.batch_size
for step in range(STEP_SIZE_TEST):
    if(step%50==0):
        print('step ', step, ' in ',  STEP_SIZE_TEST)
    x,y = test_gen.__getitem__(step)
    yh=model_final.predict(x)
    for index in range(len(y)):
        conf[np.argmax(y[index])][np.argmax(yh[index])]+=1


step  0  in  2686
step  50  in  2686
step  100  in  2686
step  150  in  2686
step  200  in  2686
step  250  in  2686
step  300  in  2686
step  350  in  2686
step  400  in  2686
step  450  in  2686
step  500  in  2686
step  550  in  2686
step  600  in  2686
step  650  in  2686
step  700  in  2686
step  750  in  2686
step  800  in  2686
step  850  in  2686
step  900  in  2686
step  950  in  2686
step  1000  in  2686
step  1050  in  2686
step  1100  in  2686
step  1150  in  2686
step  1200  in  2686
step  1250  in  2686
step  1300  in  2686
step  1350  in  2686
step  1400  in  2686
step  1450  in  2686
step  1500  in  2686
step  1550  in  2686
step  1600  in  2686
step  1650  in  2686
step  1700  in  2686
step  1750  in  2686
step  1800  in  2686
step  1850  in  2686
step  1900  in  2686
step  1950  in  2686
step  2000  in  2686
step  2050  in  2686
step  2100  in  2686
step  2150  in  2686
step  2200  in  2686
step  2250  in  2686
step  2300  in  2686
step  2350  in  2686
step  2400  in 

In [ ]:
conf

In [13]:
import csv

with open("CNN_Partial_Confision_10000_0.45.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(conf)

In [6]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../Preprocessing')
from imgDataGenerator import *
from preprocessing import *
testPath='/home/livelab/Desktop/NewResizedBinned/Test'
testGen=imgDataGenerator(testPath, shuffle=False)

/home/livelab/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [7]:
#compute confusion matrix for majority vote
#col : truth, #row : pred
conf=np.zeros((24,24))
STEP_SIZE_TEST=len(testGen.labels)//testGen.batch_size
for step in range(STEP_SIZE_TEST):
    if(step%50==0):
        print('step ', step, ' in ',  STEP_SIZE_TEST)
    x,y = testGen.__getitem__(step)
    
    yh=model_final.predict(x[0])
    major=np.zeros(24)
    for eachYh in yh:
        major=major+eachYh
    conf[np.argmax(y)][np.argmax(major)]+=1
#     for index in range(len(y)):
#         conf[np.argmax(y[index])][np.argmax(yh[index])]+=1

step  0  in  1057
step  50  in  1057
step  100  in  1057
step  150  in  1057
step  200  in  1057
step  250  in  1057
step  300  in  1057
step  350  in  1057
step  400  in  1057
step  450  in  1057
step  500  in  1057
step  550  in  1057
step  600  in  1057
step  650  in  1057
step  700  in  1057
step  750  in  1057
step  800  in  1057
step  850  in  1057
step  900  in  1057
step  950  in  1057
step  1000  in  1057
step  1050  in  1057


In [8]:
import csv

with open("model_partial_major_5k_0.50.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(conf)